<a href="https://colab.research.google.com/github/hedgehog-genki/ajax_app/blob/master/golfposerscrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## グーグルドライブのマウント
実行するとURLが表示されるので、アクセスしてワンタイムキーを取得して、入力してください。

### 実行後、対象のURLリストをドライブに格納


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


スクリプトの実行

In [ ]:
#必要なライブラリのインストール
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install requests-html

#必要なライブラリのインポート
import cv2
import numpy as np
import pandas as pd
import datetime
from tqdm.notebook import tqdm
import requests
from bs4 import BeautifulSoup
import time
import re
from urllib.request import urlopen
import urllib.request, urllib.error
from requests_html import HTMLSession
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import csv
import pprint

#スクレイピング対象のリストをインポートする
with open("/content/在庫確認納品用 - 出力用 (6).csv") as f: #ファイルパスを取得して貼り付け
  reader = csv.reader(f)
  l = [row for row in reader]

l_str = [str(n) for n in l]
#l_strがスクレイピング対象のURLが格納されるリストになります。

#スクレイピングのスクリプト
#実行するためのスクリプト(変更することはありません)
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(10)

#スクレイピングで取得した情報を格納するリストの宣言
price_list = []
select_list = []
title_list = []

#l_strリストに格納されたURLに順番にアクセスしてスクレイピングを実行
for n in l_str:
  #URLにアクセスできるように文字列を置き換える
  url4=n
  url3=url4.replace("'","")
  url2=url3.replace("[","")
  url=url2.replace("]","")
  #URLにアクセス
  driver.get(url)
  html = driver.page_source.encode('utf-8')
  #BeautifulSoupを使用してHTML要素を取得する
  soup = BeautifulSoup(html,"html.parser")
  #価格情報を取得して文字列を整理してからprice_listに格納する
  testprice = soup.find("div",attrs={"class":"product-info-price"})
  preprice = testprice.text
  rpreprice = preprice.replace("\n","")
  price = rpreprice.replace("As low as","")
  price_list.append(price)
  #タイトルの取得
  title = soup.find("h1",attrs={"class":"page-title"})
  titletext = title.text
  title_list.append(titletext)
  #在庫情報の取得
  testselect = soup.find_all("select",attrs={"super-attribute-select"})
  #取得した在庫情報をselect_listに格納する
  for s in testselect:
    pre = s.text
    pretext = pre.replace("\nSelect Size","")
    select_list.append(pretext)

#取得した価格情報と在庫情報をCSVで出力する
df_price = pd.DataFrame(price_list)
df_select = pd.DataFrame(select_list)
df_title = pd.DataFrame(title_list)
df = pd.concat([df_title,df_price,df_select],axis=1)
df.columns = ["title","price","stock"]
df.to_csv("stockinfo.csv")

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [ ]:
price_list

['$40.00',
 '$130.00',
 '$70.00',
 '$200.00',
 '$35.00',
 '$35.00',
 '$35.00',
 ' Was$94.00$66.00',
 '$66.00',
 '$110.00',
 ' Was$125.00$88.00']